<a href="https://colab.research.google.com/github/iamanujkumar/Natural-Language-Processing/blob/main/Currency_Conversion_ToolCalling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q langchain-openai langchain-core requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 8.8 MB/s eta 0:00:00


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [32]:
from langchain_core.tools import InjectedToolArg

In [33]:
from typing import Annotated
@tool
def get_conversion_factor(base_currency:str , target_currency:str)->float:
  """
  This funtion fetches the currency conversion factor between a given bace currency to a target curency
  """

  url = f"https://v6.exchangerate-api.com/v6/6d66999b1a318fa98d27cf7a/pair/{base_currency}/{target_currency}"

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value:int, conversion_rate: Annotated[float, InjectedToolArg])-> float:
  """
  gives a currency conversion rate this function calculate the target currency value from the base currency value
  """
  return base_currency_value * conversion_rate

In [34]:
get_conversion_factor.invoke({'base_currency':'USD', 'target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1758499201,
 'time_last_update_utc': 'Mon, 22 Sep 2025 00:00:01 +0000',
 'time_next_update_unix': 1758585601,
 'time_next_update_utc': 'Tue, 23 Sep 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 88.1745}

In [35]:
convert.invoke({'base_currency_value':10, 'conversion_rate':88.1745})

881.7449999999999

In [16]:
%pip install -q langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 3.9 MB/s eta 0:00:00


In [19]:
from langchain_groq import ChatGroq
from google.colab import userdata

In [36]:
llm = ChatGroq(
    groq_api_key=userdata.get("GROQ_API_KEY"),
    model="llama-3.1-8b-instant",
    temperature=0,
    )

In [37]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [38]:
messages = [HumanMessage('What is the conversion factor between USD and INR, and based on that you conver 10 usd to inr')]

In [39]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that you conver 10 usd to inr', additional_kwargs={}, response_metadata={})]

In [40]:
ai_message = llm_with_tools.invoke(messages)

In [42]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': '3g9hwqe3e',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency': 'USD',
   'base_currency_value': 10,
   'conversion_factor': 0.0065,
   'target_currency': 'INR'},
  'id': 'whjm9nser',
  'type': 'tool_call'}]

In [44]:
import json

for tool_call in ai_message.tool_calls:
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    messages.append(tool_message1)
  if tool_call['name'] == 'convert':
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [53]:
llm_with_tools.invoke(messages).content

'The conversion factor between USD and INR is 88.1745. Converting 10 USD to INR gives a value of 881.7449999999999 INR.'